## Domo arigato, Mr. Roboto

In [122]:
from pathlib import Path
import numpy as np
import re
import sys
import math

In [2]:
pattern = 'Blueprint (\d+): Each ore robot costs (\d+) ore. Each clay robot costs (\d+) ore.'
pattern +=' Each obsidian robot costs (\d+) ore and (\d+) clay. Each geode robot costs (\d+) ore and (\d+) obsidian.'

pattern = re.compile(pattern)

In [4]:
blueprints = [[int(num) for num in re.match(pattern, blueprint).groups()] 
              for blueprint in Path('blueprints_test.txt').read_text().split('\n')]

In [166]:
class Inventory():
    
    def __init__(self, blueprint, ore_robo = 1, clay_robo = 0, obs_robo = 0, 
                 geo_robo = 0, ore = 0, clay = 0, obs = 0, geo = 0, 
                 time = 0, time_limit = 24, prev_inv = None):
        
        self.blueprint = blueprint
        
        self.ore_robo_cost = blueprint[1]
        self.clay_robo_cost = blueprint[2]
        self.obsidian_robo_cost = blueprint[3:5]
        self.geode_robo_cost = blueprint[5:]
        
        
        
        self.max_ore_cost = max(map(blueprints[0].__getitem__, [1, 2, 3, 5]))
        
        
        self.ore_robo = ore_robo
        self.clay_robo = clay_robo
        self.obs_robo = obs_robo
        self.geo_robo = geo_robo
        
        self.ore = ore
        self.clay = clay
        self.obs = obs
        self.geo = geo
        
        self.time = time
        self.time_limit = time_limit
        
        self.prev_inv = prev_inv
        
    def mine(self, turns = 1):
        
        return Inventory(self.blueprint, self.ore_robo, self.clay_robo, self.obs_robo, self.geo_robo, 
                         self.ore + turns*self.ore_robo, self.clay + turns*self.clay_robo, self.obs + turns*self.obs_robo, 
                         self.geo + turns*self.geo_robo, self.time+turns)
        
    def build_options(self):
        options = []
        
        # Logic build a robot or wait until you have enough resources to build a robot you cannot build now
        
        # Do not build a robot if there are enough of that type such that the resource/minute is enough to build
        # any type of robot that requires that resource
        
        # Wait logic: Look at current income and see what types of robot can be built in the future
        # assuming no change in resource production
        
        # If there are robots within that set that are not buildable now, we can wait
        # Otherwise, just build the damn robot, Shinji
        
        
        # Check resource availability and prevent overbuilding robots
        if self.ore >= self.ore_robo_cost and self.ore_robo < self.max_ore_cost:
            options += ['ore']
        if self.ore >= self.clay_robo_cost and self.clay_robo < self.obsidian_robo_cost[1]:
            options += ['clay']
            
        if self.ore >= self.obsidian_robo_cost[0] and self.clay >= self.obsidian_robo_cost[1] and self.obs_robo < self.geode_robo_cost[1]:
            options += ['obsidian']
            
        if self.ore >= self.geode_robo_cost[0] and self.obs >= self.geode_robo_cost[1]:
            options += ['geode']
            
        
        # Look at future options, ore and clay is always available
        future_options = ['ore', 'clay']
        
        if self.clay_robo > 0:
            future_options += ['obsidian']
        if self.obs_robo > 0:
            future_options += ['geode']
        
        # Robos that we need to wait to get availalbe resources
        wait_robos = np.setdiff1d(future_options, options)
        
        if len(wait_robos) > 0:
            options += ['wait']
            
        return options
    
    def build(self, robo, prev = None):
        
        if robo == 'ore':
            return Inventory(self.blueprint, self.ore_robo+1, self.clay_robo, self.obs_robo, 
                             self.geo_robo, self.ore - self.ore_robo_cost, self.clay, self.obs, self.geo, 
                             self.time, self.time_limit, prev)
        elif robo == 'clay':
            return Inventory(self.blueprint, self.ore_robo, self.clay_robo+1, self.obs_robo, 
                             self.geo_robo, self.ore - self.clay_robo_cost, self.clay, self.obs, self.geo, 
                             self.time, self.time_limit, prev)
        elif robo == 'obsidian':
            ore_cost, clay_cost = self.obsidian_robo_cost
            
            return Inventory(self.blueprint, self.ore_robo, self.clay_robo, self.obs_robo+1, 
                             self.geo_robo, self.ore - ore_cost, self.clay - clay_cost, self.obs, self.geo, 
                             self.time, self.time_limit, prev)
        elif robo == 'geode':
            ore_cost, obs_cost = self.geode_robo_cost
            
            return Inventory(self.blueprint, self.ore_robo, self.clay_robo, self.obs_robo, 
                             self.geo_robo+1, self.ore - ore_cost, self.clay, self.obs - obs_cost, self.geo, 
                             self.time, self.time_limit, prev)
        
    def generate_children(self):
        options = []
        
        if self.ore_robo < self.max_ore_cost:
            options += ['ore']
        if self.clay_robo < self.obsidian_robo_cost[1]:
            options += ['clay']     
        if self.clay_robo > 0 and self.obs_robo < self.geode_robo_cost[1]:
            options += ['obsidian']        
        if self.obs_robo > 0 and self.obs >= self.geode_robo_cost[1]:
            options += ['geode']
        
        build_times = [self.time_to_build(robo) for robo in options]
        
        children = [self.mine(turns+1).build(robo, self) for robo, turns in zip(options, build_times) if self.time + turns + 1 <= self.time_limit]
        
        
        return children
    
    def time_to_build(self, robo):
        
        if robo == 'ore':
            return max(0, math.ceil((self.ore_robo_cost - self.ore)/self.ore_robo))
        
        elif robo == 'clay':
            return max(0, math.ceil((self.clay_robo_cost - self.ore)/self.ore_robo))
        
        elif robo == 'obsidian':
            ore_turns = math.ceil((self.obsidian_robo_cost[0] - self.ore)/self.ore_robo)
            clay_turns = math.ceil((self.obsidian_robo_cost[1] - self.clay)/self.clay_robo)
            
            return max(0, ore_turns, clay_turns)
        
        elif robo == 'geode':
            ore_turns = math.ceil((self.geode_robo_cost[0] - self.ore)/self.ore_robo)
            obs_turns = math.ceil((self.geode_robo_cost[1] - self.obs)/self.obs_robo)
            
            return max(0, ore_turns, obs_turns)
        
    def __repr__(self):
        resources = f'Ore: {self.ore}, Clay: {self.clay}, Obsidian: {self.obs}, Geode: {self.geo}'
        robots = f'Ore Robot: {self.ore_robo}, Clay Robot: {self.clay_robo}, '
        robots += f'Obsidian Robot: {self.obs_robo}, Geode Robot: {self.geo_robo}'
        turn = f'{self.time} minute' + ('s' if self.time != 1 else '')
        
        return turn + '\n' + resources + '\n' + robots
    
    def __str__(self):
        return self.__repr__()
    
    def __eq__(self, other):
        
        props = np.array(
            [self.ore_robo_cost, self.clay_robo_cost, *self.obsidian_robo_cost, 
             *self.geode_robo_cost, self.ore_robo, self.clay_robo, self.obs_robo, 
             self.geo_robo, self.ore, self.clay, self.obs, self.geo, self.time, self.time_limit]
        )
        
        other_props = np.array(
            [other.ore_robo_cost, other.clay_robo_cost, *other.obsidian_robo_cost, 
             *other.geode_robo_cost, other.ore_robo, other.clay_robo, other.obs_robo, 
             other.geo_robo, other.ore, other.clay, other.obs, other.geo, other.time, other.time_limit]
        )
        
        return np.all(props == other_props)
    
    def min_projection(self):
        time_left = (self.time_limit - self.time)
        return time_left*self.geo_robo
    
    def max_projection(self):
        time_left = (self.time_limit - self.time)
        return time_left*(self.geo_robo + self.geo_robo + time_left - 1)/2

In [97]:
def calculate_quality(blueprint, time_limit = 24):
    blueprint_num = blueprint[0]
    
    current_time = 0
    
    start_inventory = Inventory(blueprint, ore_robo = 1)
    
    status = [start_inventory]
    
    while current_time < time_limit:
        sys.stdout.write(f'\r{current_time}, {len(status):<20}')
        new_inventories = []
        
        best_min_projection = max([inventory.min_projection(time_limit-current_time) for inventory in status])
        
        for inventory in status:
            if inventory.max_projection(time_limit-current_time) < best_min_projection:
                continue
            
            build_options = inventory.build_options()
            
            inventory2 = inventory.mine()
            
            add_robo = [inventory2.build(robo) for robo in build_options]
            new_inventories += [add for add in add_robo if add not in new_inventories]
                
        status = new_inventories
        
        current_time += 1
    
    return status

In [114]:
blueprints[0]

[1, 4, 2, 3, 14, 2, 7]

In [220]:
class inventory_tree():
    
    def __init__(self, blueprint):
        
        self.start_node = Inventory(blueprint)
        
        self.best_so_far = 0
        
        self.unvisited_nodes = [start_node]
        
        self.visited_nodes = []
        
        self.final_nodes = []
        
    def dfs(self):
        while len(self.unvisited_nodes) > 0:
            
            # sys.stdout.write(f'\r{len(self.visited_nodes):<10}')
            
            target_node = self.unvisited_nodes[-1]
            self.visited_nodes += [target_node]
            self.unvisited_nodes.pop(-1)
            
            if target_node.max_projection() < self.best_so_far: # Don't pursue this branch    
                continue
            
            else:
                children = target_node.generate_children()
                
                if len(children) == 0:
                    self.final_nodes += [target_node]
                    self.best_so_far = max(self.best_so_far, target_node.min_projection())
                else:
                    self.unvisited_nodes += children
                
                
            
        

In [221]:
tree = inventory_tree(blueprints[0])
tree.dfs()

print(tree.best_so_far)

2


In [226]:
[node for node in tree.visited_nodes if node.min_projection() == 6][0]

18 minutes
Ore: 2, Clay: 12, Obsidian: 2, Geode: 0
Ore Robot: 1, Clay Robot: 4, Obsidian Robot: 2, Geode Robot: 1

In [222]:
tree.final_nodes

[24 minutes
 Ore: 10, Clay: 5, Obsidian: 6, Geode: 0
 Ore Robot: 1, Clay Robot: 4, Obsidian Robot: 2, Geode Robot: 0,
 24 minutes
 Ore: 8, Clay: 7, Obsidian: 6, Geode: 0
 Ore Robot: 1, Clay Robot: 5, Obsidian Robot: 2, Geode Robot: 0,
 24 minutes
 Ore: 6, Clay: 8, Obsidian: 6, Geode: 0
 Ore Robot: 1, Clay Robot: 6, Obsidian Robot: 2, Geode Robot: 0,
 24 minutes
 Ore: 7, Clay: 22, Obsidian: 6, Geode: 0
 Ore Robot: 1, Clay Robot: 7, Obsidian Robot: 1, Geode Robot: 0,
 24 minutes
 Ore: 5, Clay: 22, Obsidian: 6, Geode: 0
 Ore Robot: 2, Clay Robot: 6, Obsidian Robot: 1, Geode Robot: 0,
 24 minutes
 Ore: 5, Clay: 7, Obsidian: 6, Geode: 0
 Ore Robot: 2, Clay Robot: 5, Obsidian Robot: 2, Geode Robot: 0,
 24 minutes
 Ore: 6, Clay: 21, Obsidian: 6, Geode: 0
 Ore Robot: 2, Clay Robot: 6, Obsidian Robot: 1, Geode Robot: 0,
 24 minutes
 Ore: 4, Clay: 21, Obsidian: 6, Geode: 0
 Ore Robot: 3, Clay Robot: 5, Obsidian Robot: 1, Geode Robot: 0,
 24 minutes
 Ore: 8, Clay: 5, Obsidian: 6, Geode: 0
 Ore Ro

In [184]:
test.prev_inv = test

In [186]:
test.prev_inv += [test]

TypeError: unsupported operand type(s) for +=: 'Inventory' and 'list'

In [99]:
import time

In [102]:
t1 = time.time()

final_inventories = calculate_quality(blueprints[0], time_limit = 24)

t2 = time.time()

print(t2 - t1)
print(max([inventory.geo for inventory in final_inventories]))

23, 77586               350.8796844482422
8


In [108]:
blueprints[0]

[1, 4, 2, 3, 14, 2, 7]

In [107]:
final_inventories

[Ore: 3, Clay: 36, Obsidian: 8, Geode: 6
 Ore Robot: 2, Clay Robot: 8, Obsidian Robot: 4, Geode Robot: 3,
 Ore: 2, Clay: 22, Obsidian: 8, Geode: 6
 Ore Robot: 2, Clay Robot: 7, Obsidian Robot: 5, Geode Robot: 3,
 Ore: 5, Clay: 36, Obsidian: 8, Geode: 6
 Ore Robot: 2, Clay Robot: 7, Obsidian Robot: 4, Geode Robot: 3,
 Ore: 3, Clay: 35, Obsidian: 8, Geode: 6
 Ore Robot: 2, Clay Robot: 8, Obsidian Robot: 4, Geode Robot: 3,
 Ore: 2, Clay: 21, Obsidian: 8, Geode: 6
 Ore Robot: 2, Clay Robot: 7, Obsidian Robot: 5, Geode Robot: 3,
 Ore: 5, Clay: 35, Obsidian: 8, Geode: 6
 Ore Robot: 2, Clay Robot: 7, Obsidian Robot: 4, Geode Robot: 3,
 Ore: 3, Clay: 35, Obsidian: 8, Geode: 6
 Ore Robot: 2, Clay Robot: 8, Obsidian Robot: 4, Geode Robot: 3,
 Ore: 2, Clay: 21, Obsidian: 8, Geode: 6
 Ore Robot: 2, Clay Robot: 7, Obsidian Robot: 5, Geode Robot: 3,
 Ore: 5, Clay: 35, Obsidian: 8, Geode: 6
 Ore Robot: 2, Clay Robot: 7, Obsidian Robot: 4, Geode Robot: 3,
 Ore: 3, Clay: 48, Obsidian: 8, Geode: 6
 Ore 

In [72]:
print(Inventory(blueprints[0]))

Ore: 0, Clay: 0, Obsidian: 0, Geode: 0
Ore Robot: 1, Clay Robot: 0, Obsidian Robot: 0, Geode Robot: 0


In [42]:
Inventory(blueprints[0]).mine() == Inventory(blueprints[0]).mine()

True

In [36]:
[4, *Inventory(blueprints[0]).obsidian_robo_cost]

[4, 3, 14]

In [47]:
calculate_quality(blueprints[0], 24)

0 1
1 1
2 2
3 3
4 7
5 12
6 29
7 57
8 143
9 320
10 846
11 2202
12 6237
13 18267
14 56085
15 179588
16 591458
17 1997518


KeyboardInterrupt: 